# 탐색적 데이터 분석 (Exploratory Data Analysis)

이 노트북에서는 데이터를 탐색하고 이해하는 과정을 수행합니다.

## 목표
- 데이터의 구조와 특성 파악
- 결측값 및 이상치 분석
- 변수 간의 상관관계 분석
- 시각화를 통한 패턴 발견

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

print("✅ 라이브러리 임포트 완료")

: 

## 1. 데이터 로드 및 기본 정보

In [ ]:
# 샘플 데이터 생성 (실제 프로젝트에서는 데이터 로드)
from sklearn.datasets import load_iris, load_wine, load_breast_cancer

# 예시: Iris 데이터셋 사용
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df['target_name'] = df['target'].map({i: name for i, name in enumerate(data.target_names)})

print(f"데이터 형태: {df.shape}")
print(f"특성 수: {len(df.columns)-2}")
print(f"클래스 수: {df['target'].nunique()}")

In [ ]:
# 데이터 미리보기
print("=== 데이터 미리보기 ===")
display(df.head(10))

print("\n=== 데이터 타입 ===")
display(df.dtypes)

print("\n=== 기본 통계 ===")
display(df.describe())

## 2. 데이터 품질 분석

In [ ]:
# 결측값 분석
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    '결측값 개수': missing_data,
    '결측값 비율(%)': missing_percentage
})

print("=== 결측값 분석 ===")
display(missing_df[missing_df['결측값 개수'] > 0])

if missing_df['결측값 개수'].sum() == 0:
    print("✅ 결측값이 없습니다.")

In [ ]:
# 중복값 분석
duplicate_count = df.duplicated().sum()
print(f"중복 행 수: {duplicate_count}")

if duplicate_count > 0:
    print("\n중복된 행:")
    display(df[df.duplicated()])
else:
    print("✅ 중복된 행이 없습니다.")

## 3. 타겟 변수 분석

In [ ]:
# 타겟 변수 분포
target_counts = df['target_name'].value_counts()
target_percentages = df['target_name'].value_counts(normalize=True) * 100

print("=== 타겟 변수 분포 ===")
for class_name, count in target_counts.items():
    percentage = target_percentages[class_name]
    print(f"{class_name}: {count}개 ({percentage:.1f}%)")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 막대 그래프
target_counts.plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('타겟 변수 분포')
axes[0].set_xlabel('클래스')
axes[0].set_ylabel('개수')
axes[0].tick_params(axis='x', rotation=45)

# 파이 차트
axes[1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%')
axes[1].set_title('타겟 변수 비율')

plt.tight_layout()
plt.show()

## 4. 특성 변수 분포 분석

In [ ]:
# 수치형 변수 분포 히스토그램
numeric_columns = df.select_dtypes(include=[np.number]).columns.drop(['target'])
n_cols = 2
n_rows = (len(numeric_columns) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

for i, col in enumerate(numeric_columns):
    if i < len(axes):
        df[col].hist(bins=30, ax=axes[i], alpha=0.7, color='lightblue', edgecolor='black')
        axes[i].set_title(f'{col} 분포')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('빈도')

# 빈 subplot 제거
for i in range(len(numeric_columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# 박스플롯으로 이상치 탐지
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

for i, col in enumerate(numeric_columns):
    if i < len(axes):
        sns.boxplot(data=df, y=col, ax=axes[i])
        axes[i].set_title(f'{col} 박스플롯')

# 빈 subplot 제거
for i in range(len(numeric_columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

## 5. 상관관계 분석

In [ ]:
# 상관계수 행렬
correlation_matrix = df[numeric_columns].corr()

# 히트맵으로 시각화
plt.figure(figsize=(12, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            fmt='.2f')
plt.title('특성 간 상관관계 히트맵')
plt.tight_layout()
plt.show()

# 높은 상관관계 쌍 찾기
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:  # 0.7 이상의 상관관계
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                corr_value
            ))

if high_corr_pairs:
    print("\n=== 높은 상관관계 (|r| > 0.7) ===")
    for var1, var2, corr in high_corr_pairs:
        print(f"{var1} ↔ {var2}: {corr:.3f}")
else:
    print("\n높은 상관관계를 가진 변수 쌍이 없습니다.")

## 6. 클래스별 특성 분석

In [ ]:
# 클래스별 특성 분포 비교
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

for i, col in enumerate(numeric_columns):
    if i < len(axes):
        sns.violinplot(data=df, x='target_name', y=col, ax=axes[i])
        axes[i].set_title(f'{col} - 클래스별 분포')
        axes[i].tick_params(axis='x', rotation=45)

# 빈 subplot 제거
for i in range(len(numeric_columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# 클래스별 통계 요약
print("=== 클래스별 특성 통계 ===")
class_stats = df.groupby('target_name')[numeric_columns].agg(['mean', 'std'])
display(class_stats.round(3))

## 7. 인터랙티브 시각화

In [ ]:
# Plotly를 사용한 인터랙티브 산점도 행렬
feature_cols = numeric_columns[:4]  # 처음 4개 특성만 사용

fig = px.scatter_matrix(
    df, 
    dimensions=feature_cols,
    color='target_name',
    title='인터랙티브 산점도 행렬'
)

fig.update_layout(height=800)
fig.show()

In [ ]:
# 3D 산점도 (처음 3개 특성 사용)
if len(numeric_columns) >= 3:
    fig = px.scatter_3d(
        df, 
        x=numeric_columns[0], 
        y=numeric_columns[1], 
        z=numeric_columns[2],
        color='target_name',
        title='3D 산점도',
        hover_data=numeric_columns
    )
    
    fig.update_layout(height=600)
    fig.show()

## 8. 주요 발견사항 요약

In [ ]:
# 분석 결과 요약
print("=== 탐색적 데이터 분석 요약 ===")
print(f"📊 데이터 크기: {df.shape[0]:,}행 × {df.shape[1]}열")
print(f"🎯 타겟 클래스: {df['target'].nunique()}개")
print(f"📈 수치형 특성: {len(numeric_columns)}개")
print(f"❌ 결측값: {df.isnull().sum().sum()}개")
print(f"🔄 중복값: {duplicate_count}개")

# 클래스 불균형 확인
class_balance = df['target_name'].value_counts()
min_class_ratio = class_balance.min() / class_balance.max()
print(f"⚖️ 클래스 균형도: {min_class_ratio:.2f} (1.0에 가까울수록 균형)")

if min_class_ratio < 0.5:
    print("   ⚠️ 클래스 불균형이 존재합니다. 리샘플링을 고려하세요.")
else:
    print("   ✅ 클래스가 비교적 균형적입니다.")

print("\n=== 다음 단계 권장사항 ===")
print("1. 데이터 전처리 (스케일링, 인코딩)")
print("2. 피처 엔지니어링")
print("3. 특성 선택")
print("4. 모델 훈련 및 평가")